In [17]:
import os
import time
import math
import pickle as pkl

MAX_FILES = 100
REPORT_STEP = 100
WINDOW = 3
MIN_LEN_TIMELINE = 5
PAIRS_MONITOR = {('die','explode'),('attack','die'),('ask','help'),('chop','taste'),('concern','protect'),\
                 ('conspire','kill'),('debate','vote'),('dedicate','promote'),('fight','overthrow'),\
                ('achieve','desire'),('admire','respect'),('clean','contaminate'),('defend','accuse'),\
                ('die','crash'),('overthrow','elect')}

In [39]:
import numpy as np
def test():
    confusion=np.array([[1,2,3,4],[5,6,7,8],[9,0,1,2],[3,4,5,6]])
    return 1.0*np.sum([confusion[i][i] for i in range(4)])/np.sum(confusion),confusion
a=[]
b,_ = test()
c,_ = test()
a.append(float(b))
a.append(float(c))
print(a)
i,m = max(a)
print(i,m)


[0.21212121212121213, 0.21212121212121213]


TypeError: 'float' object is not iterable

In [47]:
a={'a':1,'b':2}
print(a)

{'a': 1, 'b': 2}


In [46]:
tBigram = pkl.load(open('/home/qning2/Servers/root/shared/preprocessed/qning2/temporal/TimeLines/temporal_bigram_stats.pkl','rb'))

In [19]:
tBigram.snapshot(PAIRS_MONITOR)

-------------temporal_bigram: basic stats-------------
Length of vocab: 12814
Relation set: {'before', 'after'}
Total bigrams added: 1246301
Total counts added: 13491726
------------------------------------------------------
	Pairs (PBefore)     	TotalCnt  	TBefore   	TAfter    
dedicate  	promote   	14        	10    (71.43%)		4     (28.57%)
promote   	dedicate  	0         	0     (0.00 %)		0     (0.00 %)
conspire  	kill      	10        	8     (80.00%)		2     (20.00%)
kill      	conspire  	2         	2     (100.00%)		0     (0.00 %)
attack    	die       	32        	28    (87.50%)		4     (12.50%)
die       	attack    	34        	26    (76.47%)		8     (23.53%)
ask       	help      	216       	138   (63.89%)		78    (36.11%)
help      	ask       	90        	82    (91.11%)		8     (8.89 %)
admire    	respect   	12        	12    (100.00%)		0     (0.00 %)
respect   	admire    	16        	12    (75.00%)		4     (25.00%)
overthrow 	elect     	6         	0     (0.00 %)		6     (100.00%)
elect     	ov

In [11]:
class temporal_bigram:
    def __init__(self):
        self.bigram_rel_counts = {}
        self.bigram_counts = {}
        self.vocab = set()
        self.relationSet = set(['before','after'])
        self.total_counts = 0
        self.total_bigrams = 0
    def save(self,path):
        pkl.dump(self,open(path,'wb'))
    def addWord(self,word):
        if word not in self.vocab:
            self.vocab.add(word)
    def addRelation(self,rel):
        if rel not in self.relationSet:
            self.relationSet.add(rel)
    def addOneRelation(self,v1,v2,rel):
        self.addWord(v1)
        self.addWord(v2)
        self.addRelation(rel)
        if v1 not in self.bigram_rel_counts:
            self.bigram_rel_counts[v1] = {}
            self.bigram_counts[v1] = {}
        if v2 not in self.bigram_rel_counts[v1]:
            self.bigram_rel_counts[v1][v2] = {}
            self.bigram_counts[v1][v2] = 0
            self.total_bigrams += 1
        if rel not in self.bigram_rel_counts[v1][v2]:
            self.bigram_rel_counts[v1][v2][rel] = 0
        self.bigram_rel_counts[v1][v2][rel] += 1
        self.bigram_counts[v1][v2] += 1
        self.total_counts += 1
    def getBigramCounts(self,v1,v2):
        if v1 not in self.bigram_counts or v2 not in self.bigram_counts[v1]:
            return 0
        return self.bigram_counts[v1][v2]
    def getBigramRelCounts(self,v1,v2,rel):
        if v1 not in self.bigram_rel_counts or v2 not in self.bigram_rel_counts[v1] or rel not in self.bigram_rel_counts[v1][v2]:
            return 0
        return self.bigram_rel_counts[v1][v2][rel]
    def snapshot(self,pairs2monitor=None):
        print("-------------temporal_bigram: basic stats-------------")
        print("Length of vocab: %d" % len(self.vocab))
        print("Relation set: %s" % str(self.relationSet))
        print("Total bigrams added: %d" % self.total_bigrams)
        print("Total counts added: %d" % self.total_counts)
        print("------------------------------------------------------")
        if pairs2monitor is not None:
            print("\t%s\t%s\t%s\t%s" %('Pairs (PBefore)'.ljust(20),'TotalCnt'.ljust(10),'TBefore'.ljust(10),'TAfter'.ljust(10)))
            for i, eventPair in enumerate(pairs2monitor):
                v1, v2 = eventPair[0], eventPair[1]
                if v1 not in self.vocab or \
                v2 not in self.vocab or \
                v1 not in self.bigram_rel_counts or \
                v2 not in self.bigram_rel_counts[v1]:
                    continue
                print("%s\t%s\t%-10d\t%-5d (%-5.2f%%)\t\t%-5d (%-5.2f%%)" % \
                      (v1.ljust(10),v2.ljust(10),\
                       self.getBigramCounts(v1,v2),\
                       self.getBigramRelCounts(v1,v2,'before'),\
                       100.0*self.getBigramRelCounts(v1,v2,'before')/(self.getBigramCounts(v1,v2)+1e-6),\
                       self.getBigramRelCounts(v1,v2,'after'),\
                       100.0*self.getBigramRelCounts(v1,v2,'after')/(self.getBigramCounts(v1,v2)+1e-6)))
                print("%s\t%s\t%-10d\t%-5d (%-5.2f%%)\t\t%-5d (%-5.2f%%)" % \
                      (v2.ljust(10),v1.ljust(10),\
                       self.getBigramCounts(v2,v1),\
                       self.getBigramRelCounts(v2,v1,'before'),\
                       100.0*self.getBigramRelCounts(v2,v1,'before')/(self.getBigramCounts(v2,v1)+1e-6),\
                       self.getBigramRelCounts(v2,v1,'after'),\
                       100.0*self.getBigramRelCounts(v2,v1,'after')/(self.getBigramCounts(v2,v1)+1e-6)))
            for eventPair in pairs2monitor:
                v1, v2 = eventPair[0], eventPair[1]
                if v1 not in self.vocab:
                    print('%s not in vocab!' % v1)
                if v2 not in self.vocab:
                    print('%s not in vocab!' % v2)
                if v1 not in self.bigram_rel_counts or v2 not in self.bigram_rel_counts[v1]:
                    print('(%s,%s) not in bigrams' % (v1,v2))
            print("------------------------------------------------------")
            

In [12]:
class event:
    def __init__(self,lemma,idx):
        self.lemma = lemma
        self.idx = idx
def isPBefore(event1,event2):
    return event1.idx < event2.idx
tBigram = temporal_bigram()
start = time.time()

In [13]:
INPUT_DIRECTORY = "/home/qning2/Servers/root/shared/preprocessed/sssubra2/annotated-nyt-temporal/extracted"
N_FILES = 0
for root, _, files in os.walk(INPUT_DIRECTORY):
    for fname in files:
        infile = os.path.join(root,fname)
        events = []
        lines = open(infile, 'r').readlines()
        for line in lines:
            if not line.startswith('E'):
                continue
            line = line.strip().split("/")
            lemma = line[-2] # verb lemma
            idx = int(line[0][1:])
            events.append(event(lemma,idx))
        if len(events) < MIN_LEN_TIMELINE:
            continue
        N_FILES+=1
        for i, e1 in enumerate(events):
            for j in range(max(0,i-WINDOW),min(i+WINDOW+1,len(events))):
                if i == j:
                    continue
                e2 = events[j]
                if isPBefore(e1,e2): # Text: e1...e2
                    if i < j: # Time: e1...e2
                        tBigram.addOneRelation(e1.lemma,e2.lemma,'before')
                    else: # Time: e2...e1
                        tBigram.addOneRelation(e1.lemma,e2.lemma,'after')
                else: # Text: e2...e1
                    if i < j: # Time: e1...e2
                        tBigram.addOneRelation(e2.lemma,e1.lemma,'after')
                    else: # Time: e2...e1
                        tBigram.addOneRelation(e2.lemma,e1.lemma,'before')
        if(N_FILES % REPORT_STEP == 0):
            print("%d files have been processed." % N_FILES)
            tBigram.snapshot(PAIRS_MONITOR)
            tBigram.save('temporal_bigram_stats.pkl')
        if(N_FILES>MAX_FILES):
            break
    if(N_FILES>MAX_FILES):
        break
            

100 files have been processed.
-------------temporal_bigram: basic stats-------------
Length of vocab: 909
Relation set: {'before', 'after'}
Total bigrams added: 7548
Total counts added: 19536
------------------------------------------------------
	Pairs (PBefore)     	TotalCnt  	TBefore   	TAfter    
die       	explode   	2         	2     (100.00%)		0     (0.00 %)
explode   	die       	0         	0     (0.00 %)		0     (0.00 %)
(attack,die) not in bigrams
(ask,help) not in bigrams
------------------------------------------------------


In [9]:
tBigram.snapshot(PAIRS_MONITOR)

-------------temporal_bigram: basic stats-------------
Length of vocab: 6095
Relation set: {'before', 'after'}
Total bigrams added: 325086
Total counts added: 1803426
------------------------------------------------------
	Pairs (PBefore)     	TotalCnt  	TBefore   	TAfter    
attack    	die       	4         	2     (50.00%)		2     (50.00%)
die       	attack    	6         	2          (33.33     %)		4          (66.67     %)
die       	explode   	4         	4     (100.00%)		0     (0.00 %)
explode   	die       	6         	6          (100.00    %)		0          (0.00      %)
ask       	help      	22        	16    (72.73%)		6     (27.27%)
help      	ask       	12        	10         (83.33     %)		2          (16.67     %)
------------------------------------------------------
